## **IMPORTACIONES**

In [27]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## **API Key**

In [28]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if api_key:
    print("API Key cargada.")
else:
    print("No se encontró la API Key.")

API Key cargada.


### **Carga de PDFs**
- Cargamos los PDFs, separamos las páginas y le damos una fuente a cada una según al PDF que pertenece.

In [29]:
def load_and_tag_pdfs(data_dir: str, paper_sources: dict):
   
    documents = []

    for filename, source_title in paper_sources.items(): # Itera en el diccionario por cada PDF (toma el nombre del archivo y el título de la fuente) 
        file_path = os.path.join(data_dir, filename)
        
        if not os.path.exists(file_path):
            print(f"ADVERTENCIA: No se encontró el archivo {file_path}")
            continue
            
        try:
            loader = PyPDFLoader(file_path) # Carga el PDF
            pages = loader.load() # Carga las páginas del PDF
            
            for page in pages: # Etiqueta cada página con el título de la fuente (cada página de BERT.pdf tendrá la metadata "source": "BERT")
                page.metadata["source"] = source_title
                
            documents.extend(pages) # Agrega las páginas etiquetadas a la lista de documentos
            print(f"Cargado y etiquetado: {filename} ({len(pages)} páginas)")
            
        except Exception as e:
            print(f"Error al cargar {file_path}: {e}")
            
    return documents

### **Directorio y diccionario de los PDFs**
- Creamos los `parametros` que va a necesitar la función que creamos previamente.

In [30]:
# Directorio de los PDFs
DATA_FOLDER = "./docs" 

# Cada PDF y su título real (los títulos servirán para etiquetar las páginas)
PAPERS = {
    "AIAYN.pdf": "Attention Is All You Need (Vaswani et al., 2017)",
    "BERT.pdf": "BERT: Pre-training of Deep Bidirectional Transformers (Devlin et al., 2018)",
    "RAG.pdf": "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks (Lewis et al., 2020)"
}

# Llamamos a la funcion que creamos previamente pasandole el directorio y los PDFs
all_documents = load_and_tag_pdfs(DATA_FOLDER, PAPERS)

if all_documents:
    print(f"\nTotal de páginas cargadas: {len(all_documents)}")
    print(f"Fuente de la página 1: {all_documents[0].metadata['source']}")

Cargado y etiquetado: AIAYN.pdf (15 páginas)
Cargado y etiquetado: BERT.pdf (16 páginas)
Cargado y etiquetado: RAG.pdf (19 páginas)

Total de páginas cargadas: 50
Fuente de la página 1: Attention Is All You Need (Vaswani et al., 2017)


## **Chunking**
- Fragmentación del texto

In [31]:
def split_documents(documents):
    
    # Configuración del separador
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, # Traerá un máximo de 1000 caracteres por chunk
        chunk_overlap=150, # Retrocederá hasta 150 caracteres para el siguiente chunk
        separators=["\n\n", "\n", ". ", " ", ""] # Busca saltos de linea dobles, simples, puntos, etc. (hace que la separación sea más "natural" semánticamente).
    )
    
    chunks = text_splitter.split_documents(documents) # Aplica el separador a los documentos cargados
    
    print(f"Total de chunks creados: {len(chunks)}")
    
    if chunks:
        print(f"Metadatos del primer chunk: {chunks[0].metadata['source']}")
        
    return chunks

In [32]:
document_chunks = split_documents(all_documents) # Fragmentamos los documentos cargados llamando a la función previamente creada

Total de chunks creados: 220
Metadatos del primer chunk: Attention Is All You Need (Vaswani et al., 2017)


### **Embedding y Vector Store**
- Convertimos cada chunk en un vector (Embbeding) y los guardamos en una BD (Vector Store) 

In [33]:
# Conectamos con Ollama y utilizamos el modelo de embedding "nomic-embed-text"
try:
    embedding_model = OllamaEmbeddings(
        model="nomic-embed-text"
    )
    print("Modelo de embedding 'nomic-embed-text' conectado vía Ollama.")
except Exception as e:
    print(f"ERROR: No se pudo conectar a Ollama. ¿Está corriendo? {e}")

# Definimos dónde guardar la BD y cómo llamarla
PERSIST_DIRECTORY = "./chroma_db_tutor_ia"
COLLECTION_NAME = "ia_papers_tutor"

# Creamos la BD
vector_store = Chroma.from_documents(
    documents=document_chunks, # La lista de chunks que creamos en la celda anterior
    embedding=embedding_model,  # El modelo que usará para convertir chunks a vectores
    persist_directory=PERSIST_DIRECTORY, # Dónde guardar la base de datos
    collection_name=COLLECTION_NAME    # El nombre de la "tabla" dentro de la BD
)

print(f"¡Vector Store lista y guardada en {PERSIST_DIRECTORY}!")

Modelo de embedding 'nomic-embed-text' conectado vía Ollama.
¡Vector Store lista y guardada en ./chroma_db_tutor_ia!


---
## **PIPELINE**

### **Retriever**
- Búsqueda `vectores relevantes`.

In [34]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3}) # Le decimos que traiga los 3 chunks más relevantes según la pregunta realizada.

### **Formateo de contenido**
- Dividimos lo que sería la `fuente` y el `contenido` de la respuesta.

In [35]:
def format_docs_with_sources(docs):
    return "\n\n---\n\n".join(
        f"Fuente: {doc.metadata['source']}\nFragmento: {doc.page_content}"
        for doc in docs
    )

### **Prompt y su template**
- Las reglas que seguirá nuestro modelo a la hora de responder y le decimos que recibirá un contexto y la pregunta.

In [36]:
template = """
Eres un "Tutor de Investigación" experto en IA. Tu única fuente de conocimiento son los siguientes fragmentos de papers fundacionales.

REGLAS ESTRICTAS:
1. Responde la pregunta del estudiante basándote *única y exclusivamente* en el contexto proporcionado.
2. Al final de tu respuesta, DEBES citar la fuente exacta del paper que usaste (ej: "Fuente: Attention Is All You Need (Vaswani et al., 2017)").
3. Si el contexto no contiene la información para responder, DEBES responder exactamente: "Lo siento, no tengo información sobre eso en mis documentos fundacionales."

---
CONTEXTO PROPORCIONADO:
{context}
---

PREGUNTA DEL ESTUDIANTE:
{question}

RESPUESTA DEL TUTOR:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

### **Conexión con el modelo**
- El generador de respuesta.

In [37]:
llm = ChatGoogleGenerativeAI(
    api_key=api_key, # Utilizamos la api_key provista anteriormente.
    model="gemini-1.5-pro-latest", # El modelo Gemini a utilizar.
    temperature=0.3 # Baja temperatura para que sea fiel al texto.
)

### **Pipeline completo**
- Aqui `ensamblamos` todas las celdas creadas en ``Pipeline`.

In [38]:
rag_chain = (
    # Creamos un diccionario con el contexto y la pregunta
    {
        "question": RunnablePassthrough(), # La pregunta del usuario pasa tal cual.
        "context": retriever | format_docs_with_sources # Llama a retriever y luego formatea los documentos.
    }
    
    # Luego, pasamos ese diccionario al prompt
    | prompt
    
    # Luego, pasamos el prompt al LLM
    | llm
    
    # Finalmente, obtenemos la respuesta de texto
    | StrOutputParser()
)

print("Pipeline creado")  

Pipeline creado
